In [8]:
!conda install --name bananavision --file requirements.txt

Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: ...working... unsuccessful initial attempt using frozen solve. Retrying with flexible solve.


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/noarch/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/main/win-64/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/msys2/win-64/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/msys2/noarch/current_repodata.json HTTP/1.1" 304 0
DE

In [2]:
!setup_environment.sh


In [7]:
import os
import pandas as pd
from sklearn.utils import shuffle
from PIL import Image
import torch
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import sys

ModuleNotFoundError: No module named 'sklearn'

In [11]:

def create_dataframe_from_images(folders):
    """
    Create a DataFrame by reading images from multiple folders and labeling them with the folder names.

    Args:
        folders (list): List of folder paths to read images from.

    Returns:
        pandas.DataFrame: DataFrame containing image paths and labels.
    """
    data = []
    for folder in folders:
        for filename in os.listdir(folder):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                image_path = os.path.join(folder, filename)
                label = os.path.basename(folder)
                data.append((image_path, label))

    df = pd.DataFrame(data, columns=['image_path', 'label'])
    df = shuffle(df)
    return df

def preprocess_images(df, target_size=(224, 224)):
    """
    Preprocess the images in the DataFrame.

    Args:
        df (pandas.DataFrame): DataFrame containing image paths and labels.
        target_size (tuple): Target spatial size for resizing. Defaults to (224, 224).

    Returns:
        tuple: A tuple containing the preprocessed images and corresponding one-hot encoded labels.
    """
    transform = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    images = []
    labels = []

    for image_path, label in df.values:
        image = Image.open(image_path)
        image = transform(image)
        images.append(image)
        labels.append(label)

    encoded_labels = onehot_encode_labels(np.array(labels).reshape(-1, 1))  # One-hot encode labels

    return images, encoded_labels


def onehot_encode_labels(labels):
    """
    One-hot encode the labels in the list.

    Args:
        labels (list): List of labels.

    Returns:
        torch.Tensor: One-hot encoded labels.
    """
    encoder = OneHotEncoder(handle_unknown='ignore')
    encoded_labels = encoder.fit_transform(labels).toarray()
    return torch.tensor(encoded_labels)
def train_model(images, labels, num_epochs=50, batch_size=16, num_classes=3):
    """
    Train the ResNet-152 model on the given images and labels.

    Args:
        images (list of torch.Tensor): List of preprocessed images.
        labels (list of torch.Tensor): List of one-hot encoded labels.
        num_epochs (int, optional): Number of training epochs. Defaults to 10.
        batch_size (int, optional): Batch size for training. Defaults to 16.
        num_classes (int, optional): Number of classes

    Returns:
        torch.nn.Module: Trained ResNet-152 model.
    """
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = models.resnet152()  # Use ResNet-152
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)  # Customize the last fully connected layer

    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Convert lists of tensors to single tensors
    images_tensor = torch.stack(images)
    labels_tensor = torch.stack(tuple(labels))

    # Create a TensorDataset from the tensors
    train_dataset = torch.utils.data.TensorDataset(images_tensor, labels_tensor)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    model.to(device)

    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            # Print loss every 100 batches
            if i % 100 == 99:
                print(f"Epoch: {epoch + 1}/{num_epochs}, Batch: {i + 1}, Loss: {running_loss / 100}")
                running_loss = 0.0
                sys.stdout.flush()  # Force the output to be displayed

        # Print loss at the end of each epoch
        print(f"Epoch: {epoch + 1}/{num_epochs}, Final Loss: {running_loss / len(train_loader)}")
        sys.stdout.flush()
    return model

def validate_model(model, val_loader, criterion, device):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)

            # Convert one-hot encoded labels to class indices
            _, labels = torch.max(labels, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return val_loss, accuracy


                

In [12]:
# Create a DataFrame from the images in the training and validation folders
df_train = create_dataframe_from_images(['data/train/BLACK SIGATOKA_1', 'data/train/FUSARIUM WILT_1','data/train/HEALTHY_1'])
df_validation = create_dataframe_from_images(['data/test/BLACK SIGATOKA_2', 'data/test/FUSARIUM WILT_2','data/test/HEALTHY_2'])


NameError: name 'shuffle' is not defined

In [ ]:
display(df_train.head())
display(df_train.shape)
display(df_validation.head())
display(df_validation.shape)

In [ ]:
# Preprocess the images
train_images,train_labels = preprocess_images(df_train)
test_images,test_labels = preprocess_images(df_validation)


In [ ]:
# Define your validation DataLoader, criterion, and device
# Convert lists of tensors to single tensors
images_tensor = torch.stack(test_images)
labels_tensor = torch.stack(tuple(test_labels))
val_dataset = torch.utils.data.TensorDataset(images_tensor, labels_tensor)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)
criterion = nn.CrossEntropyLoss()  # Use an appropriate criterion for your task
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Train the model
trained_model = train_model(train_images, train_labels)



In [9]:
# Validate the trained model
val_loss, val_accuracy = validate_model(trained_model, val_loader, criterion, device)

print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")


,image_path,label
0,data/train/BLACK SIGATOKA_1\Image_870.jpg,BLACK SIGATOKA_1
1,data/train/BLACK SIGATOKA_1\Image_871.jpg,BLACK SIGATOKA_1
2,data/train/BLACK SIGATOKA_1\Image_872.jpg,BLACK SIGATOKA_1
3,data/train/BLACK SIGATOKA_1\Image_873.jpg,BLACK SIGATOKA_1
4,data/train/BLACK SIGATOKA_1\Image_874.jpg,BLACK SIGATOKA_1


In [9]:
from PIL import Image
import torchvision.transforms as transforms
import torch
import matplotlib.pyplot as plt

def preprocess_single_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    image = Image.open(image_path)
    image = transform(image)
    image = torch.unsqueeze(image, 0)
    return image

def predict_and_display_image(model, image_path, label_names, device):
    model.eval()
    image = preprocess_single_image(image_path)
    image = image.to(device)

    with torch.no_grad():
        output = model(image)

    probabilities = torch.nn.functional.softmax(output, dim=1)[0]  # Assuming batch size is 1
    predicted_prob, predicted_class = torch.max(probabilities, 0)
    predicted_label = label_names[predicted_class.item()]

    # Display the image
    image = Image.open(image_path)
    plt.imshow(image)
    plt.title(f"Predicted Label: {predicted_label}, Probability: {predicted_prob.item():.2%}")
    plt.axis('off')
    plt.show()

image_path_to_test = 'data/test/HEALTHY_2/Image_5904.jpg'
label_names = ['BLACK SIGATOKA', 'FUSARIUM WILT', 'HEALTHY']
predict_and_display_image(trained_model, image_path_to_test, label_names, device)


ModuleNotFoundError: No module named 'torchvision'